In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow_datasets as tfds
import tensorflow as tf

import time
import numpy as np

import pretty_midi
import glob
import os 

import pickle

In [0]:
tf.compat.v1.enable_eager_execution()

Make training data

In [0]:
def get_notes(file_path):
    notes = []
    ins = pretty_midi.PrettyMIDI(file_path)
    if len(ins.instruments) > 1:
        print(file_path, 'is not single channel')
        return None
    for i in ins.instruments:
        for note in i.notes:
            note.start = ins.time_to_tick(note.start)
            note.end = ins.time_to_tick(note.end)
        notes.append(i.notes)
    return notes

In [5]:
data_dir = '/content/drive/My Drive/CarolComposer/carol_dataset'
midi_datas = []
for x in os.walk(data_dir):
    for y in glob.glob(os.path.join(x[0], '*.mid')):
        midi_datas.append(y)

print('total_file_num:', len(midi_datas))
all_notes = []
note_index = 0
for midi_data in midi_datas:
    midi_notes = get_notes(midi_data)
    for notes in midi_notes:
        notes_to_strings = []
        for note in notes:
            notes_to_string = str(note.start) + '/' + str(note.end) + '/' + str(note.pitch)
            notes_to_strings.append(notes_to_string)
        notes_to_strings.append('0/0/0')
    all_notes.append(notes_to_strings)

print(len(all_notes))

total_file_num: 37
37


In [0]:
idx = 0
for notes in all_notes:
    for note in notes:
        idx += 1

total_note_len = idx      

In [0]:
vocabs = set()
for notes in all_notes:
    for note in notes:
        vocabs.add(note)

vocabs = list(vocabs)
# save musical vocabs
with open('/content/drive/My Drive/CarolComposer/vocabs', 'wb') as fp:
    pickle.dump(vocabs, fp)

# load saved musical vocabs
loaded_vocab = set()
with open('/content/drive/My Drive/CarolComposer/vocabs', 'rb') as fp:
    loaded_vocab = pickle.load(fp)

In [8]:
note2idx = {u:i for i, u in enumerate(loaded_vocab)}
idx2note = np.array(loaded_vocab)

print('note2idx:', note2idx)
print('idx2note:', idx2note)

song_as_int = []

for notes in all_notes:
    for note in notes:
        if note in note2idx:
            song_as_int.append(note2idx[note])

song_as_int = np.array(song_as_int)

note2idx: {'21760/22016/54': 0, '1792/2048/55': 1, '8064/8192/63': 2, '1024/1280/50': 3, '3072/3200/69': 4, '17472/17536/65': 5, '12544/12928/59': 6, '25344/25600/48': 7, '256/500/62': 8, '1024/1536/56': 9, '26368/26624/48': 10, '2560/2944/73': 11, '30208/30464/53': 12, '38144/38400/60': 13, '11648/11776/58': 14, '5248/5376/71': 15, '4608/4852/52': 16, '9472/9600/69': 17, '46848/47616/55': 18, '19968/20416/59': 19, '14720/14848/70': 20, '1024/1536/59': 21, '26880/27392/45': 22, '1536/1792/65': 23, '17408/17664/65': 24, '41472/41728/72': 25, '5120/5248/64': 26, '8704/8832/67': 27, '35328/35584/52': 28, '4864/5108/54': 29, '6784/6912/74': 30, '6400/6656/60': 31, '18944/19200/76': 32, '7424/8448/57': 33, '13568/13824/72': 34, '16896/17024/43': 35, '512/896/67': 36, '4608/4992/55': 37, '19072/19200/67': 38, '10496/10752/66': 39, '13056/13312/60': 40, '6400/6528/77': 41, '768/1536/55': 42, '23040/23168/72': 43, '9216/9472/55': 44, '23232/23360/71': 45, '16896/17024/72': 46, '17920/18048/72'

In [9]:
print(song_as_int)

[9210 5835 3206 ...  471  844 4050]


In [0]:
seq_length = 20
examples_per_epoch = total_note_len//(seq_length+1)

note_dataset = tf.data.Dataset.from_tensor_slices(song_as_int)

In [0]:
sequences = note_dataset.batch(seq_length+1, drop_remainder=True)

In [0]:
def split_input_target(chunk):
    input_note = chunk[:-1]
    target_note = chunk[1:]
    return input_note, target_note

dataset = sequences.map(split_input_target)

In [0]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

Build model

In [0]:
vocab_size = len(vocabs)
embedding_dim = 256
rnn_units = 512

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
      tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
      tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
    vocab_size = len(vocabs),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           2476288   
_________________________________________________________________
gru (GRU)                    (64, None, 512)           1181184   
_________________________________________________________________
dense (Dense)                (64, None, 9673)          4962249   
Total params: 8,619,721
Trainable params: 8,619,721
Non-trainable params: 0
_________________________________________________________________


In [0]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [0]:
my_adam = tf.keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-6, amsgrad=False)
model.compile(optimizer=my_adam, loss=loss)

In [0]:
EPOCHS=20

In [0]:
checkpoint_dir = '/content/drive/My Drive/CarolComposer/model_output'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [22]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
9/9 [==============================] - 3s 330ms/step - loss: 9.1979
Epoch 2/20
9/9 [==============================] - 1s 102ms/step - loss: 9.0685
Epoch 3/20
9/9 [==============================] - 1s 109ms/step - loss: 8.5951
Epoch 4/20
9/9 [==============================] - 1s 118ms/step - loss: 6.8838
Epoch 5/20
9/9 [==============================] - 1s 112ms/step - loss: 4.3765
Epoch 6/20
9/9 [==============================] - 1s 106ms/step - loss: 1.6627
Epoch 7/20
9/9 [==============================] - 1s 125ms/step - loss: 0.6200
Epoch 8/20
9/9 [==============================] - 1s 121ms/step - loss: 0.5105
Epoch 9/20
9/9 [==============================] - 2s 203ms/step - loss: 0.2068
Epoch 10/20
9/9 [==============================] - 1s 160ms/step - loss: 0.0698
Epoch 11/20
9/9 [==============================] - 1s 123ms/step - loss: 0.0294
Epoch 12/20
9/9 [==================

In [0]:
checkpoint_dir = '/content/drive/My Drive/CarolComposer/model_output'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
def generate_midi_tick(model):
    num_generate = 1000


    input_eval = [note2idx['0/0/0']]
    input_eval = tf.expand_dims(input_eval, 0)

    generated_music = []

    temperature = 1

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)
        generated_music.append(idx2note[predicted_id])

    return generated_music

In [0]:
genereated_midis = generate_midi_tick(model)

In [53]:
print(genereated_midis)

['0/192/65', '192/256/69', '256/512/72', '256/768/65', '512/768/72', '256/768/53', '256/768/57', '896/1024/52', '1024/1280/74', '1024/1536/64', '1024/1536/68', '1280/1536/71', '768/1536/47', '1024/1536/56', '1664/1792/53', '1792/2048/71', '5632/5888/52', '5888/6144/69', '5888/6144/62', '5888/6144/66', '5888/6144/50', '6144/6400/71', '6144/6400/62', '6144/6400/67', '6144/6400/43', '6400/6912/66', '6400/6912/57', '6400/6912/62', '6400/6912/45', '6912/7296/64', '6912/7296/64', '6912/7296/72', '7168/7296/52', '7168/7296/52', '7296/7680/67', '7296/7680/76', '7296/7680/60', '7296/7680/48', '7680/7872/64', '7680/7872/72', '7872/7936/64', '7872/7936/67', '7680/7936/52', '7680/7936/55', '7936/8064/60', '7936/8064/64', '7936/8064/48', '7936/8064/55', '8064/8256/59', '8064/8256/67', '23808/24064/64', '23808/24064/48', '23808/24064/55', '24064/24576/60', '24064/24576/65', '24064/24576/53', '24064/24576/57', '0/0/0', '0/128/67', '0/128/55', '128/256/72', '256/320/72', '128/384/64', '320/384/71', '1

In [0]:
new_music = pretty_midi.PrettyMIDI(midi_file=None, resolution=256, initial_tempo=60.0)
piano_program = pretty_midi.instrument_name_to_program('Acoustic Grand Piano')
piano = pretty_midi.Instrument(program=piano_program)

for midi in genereated_midis:
    start_sec = new_music.tick_to_time(int(midi.split('/')[0]))
    end_sec = new_music.tick_to_time(int(midi.split('/')[1]))
    pitch = int(midi.split('/')[2])
    note = pretty_midi.Note(100, pitch, start_sec, end_sec)
    piano.notes.append(note)

new_music.instruments.append(piano)

In [0]:
new_music.write('/content/drive/My Drive/CarolComposer/output_music.mid')